# Genera el árbol de todas las posiciones posibles de backgamon 

## Referencias
- https://en.wikipedia.org/wiki/Backgammon
- https://www.w3schools.com/python/
- https://www.sqlitetutorial.net/
- https://docs.python.org/3/library/sqlite3.html
- https://www.tutorialspoint.com/python_design_patterns/index.htm


In [ ]:
_STANDARD_START_ = [
        2, 0, 0, 0, 0,-5,
        0,-3, 0, 0, 0, 5,
        -5, 0, 0, 0, 3, 0,
        5, 0, 0, 0, 0, 2
]
_ZERO_STATE_ = [
    0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0,
    0,  0,  0,  0,  0,  0
]

START_POSITION = 0
END_POSITION = 23
MYSELF = 1
OPPONENT = -1
APPROACH_STAGE = 100
FINAL_RUN_STAGE = 200


class BackagmonState():



    _STANDARD_START_ = [
            2, 0, 0, 0, 0,-5,
            0,-3, 0, 0, 0, 5,
            -5, 0, 0, 0, 3, 0,
            5, 0, 0, 0, 0, 2
    ]
    _ZERO_STATE_ = [
        0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0
    ]

    START_POSITION = 0
    END_POSITION = 23
    MYSELF = 1
    OPPONENT = -1
    

    def __init__(self, tiles=_STANDARD_START_, home=[0,0], jail=[0,0]):
        self.tiles = tiles        
        self.home = home
        self.jail = jail
    
    def __str__(self):
        _tiles_ =  [str(t) for t in self.tiles]         
        _tiles_ = ",".join(_tiles_)
        _home_ =  [str(h) for h in self.home]         
        _home_ = ",".join(_home_)
        _jail_ =  [str(j) for j in self.jail]         
        _jail_ = ",".join(_jail_)
        return f"<{_tiles_}|{_home_}|{_jail_}>"
        
    def game_stage(self, turn):
        if turn == MYSELF:
            
            pass
        elif turn == OPPONENT:
            pass
        else: 
            raise Error(f'EL valor de turno es incorrecto. turno = {turno}')


    def intentar_movimiento (self, ini_position, dice, turn):
        '''
        Intenta un movimiento a partir de:
            position: Posición inicial [0..23] desde la que va a intentar hacer el movimiento. 
            dice: valor de dado [1..6] indicando las posiciones que va a mover
            turn: turno al que le corresponde mover 1=yo, -1=oponente
        
        Devuelve: 
            objeto BackgammonState si puede realizar un movimiento válido 
            None si no puede realizar el movimiento
        '''

        end_position = ini_position + (dice * turn) #if turn = -1, it is opponent turn the move is in the opposite direction
        #ini_position_checkers = count_checkers(ini_position)
        #new_postion_checkers = count_checkers(end_position)

        #Errores de software - violación de precondiciones
        if ini_position > END_POSITION or end_position < START_POSITION:
            print(f'{self}-{ini_position}-{dice}-{turn}: No es válido, dado que la posición inicial se sale de los límites')
            return None
        
        if dice < 1 or dice > 6:            
            print(f'{self}-{ini_position}-{dice}-{turn}: No es válido, dado que el valor del dado se sale de los límites')
            return None
        
                
        if turn != MYSELF and turn != OPPONENT:            
            print(f'{self}-{ini_position}-{dice}-{turn}: No es válido, dado que el turno no corresponde a ninguno de los oponentes')
            return None


        #Verificar el movimiento es válido
        if end_position > END_POSITION or end_position < START_POSITION:
            print(f'{self}-{ini_position}-{dice}-{turn}: No es válido, dado que la posición final se sale de los límites')
            return None

        #Debe haber checkers > 0 si turn = 1 y checkers < 0 si turn = -1
        if (self.tiles[ini_position] * turn) <= 0: 
            print(f'{self}-{ini_position}-{dice}-{turn}: No es válido, dado que no hay fichas que mover en la posición')
            return None

        #Si esta llegando a home, debe estar en modo FINAL_RUN


        print(f'{self}-{ini_position}-{dice}-{turn}: OK')
        

In [ ]:
estado = BackagmonState()
print(estado)
estado.intentar_movimiento(25, 2, 1)
estado.intentar_movimiento(22, 8, 1)
estado.intentar_movimiento(22, 1, 1)
estado.intentar_movimiento(0,1,1)
estado.intentar_movimiento(1,1,1)
estado.intentar_movimiento(1,1,-1)
estado.intentar_movimiento(0,1,-1)




<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>-25-2-1: No es válido, dado que la posición inicial se sale de los límites
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>-22-8-1: No es válido, dado que el valor del dado se sale de los límites
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>-22-1-1: No es válido, dado que no hay fichas que mover en la posición
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>-0-1-1: OK
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>-1-1-1: No es válido, dado que no hay fichas que mover en la posición
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>-1-1--1: No es válido, dado que no hay fichas que mover en la posición
<2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,2|0,0|0,0>-0-1--1: No es válido, dado que la posición inicial se sale de los límites
